In [ ]:
%%time
import read_flac_file as rff
import numpy as np
import pandas as pd
import iisignature

def cycle_sigs(voltage, current, deg, flag):
    
    # number of channels (dimensions) in path - time, voltage, current
    m = 2

    # find the signature length using an arbitrary path
    
    
    len_sig = iisignature.logsiglength(2, deg)
    
    #prepare log-signature
    s = iisignature.prepare(2, deg)

    # find zero up-crossing indexes for voltage
    zci = np.where(np.diff(np.sign(voltage))>0)[0]
    
    
    # downsample so that we have downsample cycles per second
    zci2 = zci[1::10]
    zci = zci[::10]
    
    if len(zci) > len(zci2):
            zci=zci[0:-1]
    #debug
        #print('Limiting zci')
        #zci = zci[0:100000]

    if flag == True:
        sigs = np.empty([len(zci)-2,len_sig]);

        # create a path for each cycle 
        for k,z in enumerate(zci[0:-2]):

            ch1 = voltage[zci[k]:zci2[k]]
            ch2 = current[zci[k]:zci2[k]]
            ch2 = np.cumsum(ch2)
            path = np.column_stack([ch1, ch2])
            #pt.plot(path)

            # create the signature using iisignature
            sigs[k,:]  = iisignature.logsig(path,s)

        # drop the two final zci because we discarded them in the for loop

        return sigs
    
    else:
        zci = zci[0:-2]
        zci2 = zci2[0:-2]
        return zci, zci2
# Done

dir_flacs = '/scratch/dale_data/UK-DALE-2017/UK-DALE-2017-16kHz/house_1/2016/wk01'

flacs = ["vi-1451865600_761652.flac",
    "vi-1451869200_223156.flac",
    "vi-1451872800_857957.flac",
    "vi-1451876400_469645.flac",
    "vi-1451880000_123879.flac",
    "vi-1451887200_202095.flac",
    "vi-1451890800_368579.flac",
    "vi-1451894400_960945.flac",
    "vi-1451898000_508684.flac",
    "vi-1451901600_056124.flac",
    "vi-1451908800_261828.flac",
    "vi-1451916000_275589.flac",
    "vi-1451919600_913377.flac",
    "vi-1451923200_442540.flac", 
    "vi-1451926800_118154.flac", 
    "vi-1451930400_688062.flac", 
    "vi-1451934000_341065.flac",
    "vi-1451937600_102178.flac",
    "vi-1451941200_589828.flac",
    "vi-1451944800_203395.flac"]

test_errors = np.empty(0)

for fn_flac in flacs:
    (voltage, current) = rff.read_flac(dir_flacs + "/" + fn_flac, True)
    (zci, zci2) = cycle_sigs(voltage, current, 2, False)

    power_per_cycle = np.empty(len(zci))
    for k in range(len(zci)):
        power_per_cycle[k] = sum(voltage[zci[k]:zci2[k]] * current[zci[k]:zci2[k]])/319
    (scaled_voltage, scaled_current) = rff.read_flac(dir_flacs + "/" + fn_flac, False)
    sigs = cycle_sigs(scaled_voltage, scaled_current, 2, True)    
    my_data = pd.DataFrame(sigs)
    my_data['power'] = pd.Series(power_per_cycle)
    features = np.arange(my_data.columns.shape[0] - 1)
    my_data = my_data.dropna()
    X, y = my_data[features], my_data['power']
    ratio = 0.9
    break_point = int(ratio * my_data.shape[0])

    X_train = X.iloc[0:break_point, :]
    y_train = y[0:break_point]
    X_test = X.iloc[break_point:, :]
    y_test = y[break_point:]

    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
    rf.fit(X_train, y_train);
    predictions = rf.predict(X_test)
    #predictions_train = rf.predict(X_train)
    errors = (abs(predictions - y_test)/abs(y_test))*100
    #errors_train = (abs(predictions_train - y_train)/y_train)*100

    test_errors = np.append(test_errors, round(np.mean(errors), 2))
    
    print("Mean relative error for testing set : ", round(np.mean(errors), 2))
    #print("Mean relative error for training set: ", round(np.mean(errors_train), 2))
    #importances = list(rf.feature_importances_)
    #feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
    #feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    #[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];
    #plt.plot(my_data['power'])
print(round(np.mean(test_errors),2))
print(round(np.std(test_errors), 2))